In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import random

## Sets up the WebDriver

In [17]:
#given a path launches a website
Path = "/Users/ramirostorni/Documents/chromedriver"
driver = webdriver.Chrome(Path)

/var/folders/48/1t2lf7js69g6my_l4x0npdrw0000gn/T/ipykernel_38412/1960765232.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(Path)


In [18]:
# driver.get("https://www.christies.com/en/results")
driver.get("https://www.christies.com/en/results?language=en&month=3&year=2013")

driver.maximize_window()

## Function to Close Cookie PopUp

In [19]:
def CookieCloser():
    try:
        cookie_event = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID ,"onetrust-accept-btn-handler"))
        )
        cookie_event.click()
    except:
        driver.quit()

## Function to Close Email PopUp

In [21]:
def ClickEmail():
    try:
        email_event = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID ,"close_signup"))
        )
        time.sleep(1)
        email_event.click()
    except:
        driver.quit()

## Function to select the catagories filter 

In [23]:
def click_catagories():
    
    button = driver.find_element(By.XPATH, '//*[text()="Category"]')
    
    driver.execute_script("arguments[0].scrollIntoView();", button)
    time.sleep(.5)
    
    button.click()

## Function to select the fine arts filter 

In [24]:
def click_fine_art():
    
    Fine_Art = driver.find_element(By.XPATH, '//*[text()="Fine Art "]')
    
    driver.execute_script("arguments[0].scrollIntoView();", Fine_Art)
    time.sleep(0.5)
    
    Fine_Art.click() 

## Function to select the prints and photographs filter 

In [25]:
def click_prints():
    
    Photographs_Prints  = driver.find_element(By.XPATH, '//*[text()="Photographs & Prints "]') 

    driver.execute_script("arguments[0].scrollIntoView();", Photographs_Prints)
    time.sleep(.5)
    
    Photographs_Prints .click()

## Function clicks on the next page button

In [26]:
def click_next_page():
    
    main = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'calendar-tab'))
        )
    
    footer = main.find_element(By.CLASS_NAME, 'calendar-list__footer')
    
    next_button = footer.find_element(By.CLASS_NAME, 'chr-button__text')
    
    time.sleep(1)
    driver.execute_script("arguments[0].scrollIntoView();", next_button)
    time.sleep(1)
    
    next_button.click()

## Features:

In [27]:
Auction_titles = []
Auction_type = []
Auction_sale_total = []
Auction_location = []
Auction_date = []
Auction_URL = []

In [28]:
def extract_General_Auction_Data():

    time.sleep(.2)
    top_of_page = driver.find_element(By.TAG_NAME, 'h3')
    time.sleep(.2)
    
    main = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "calendar-tab"))
        )
    
    time.sleep(.2)
    driver.execute_script("arguments[0].scrollIntoView();", top_of_page)
    time.sleep(.2)
    
    ul = main.find_element(By.TAG_NAME, 'ul')

    auctions = ul.find_elements(By.TAG_NAME, 'li')

    time.sleep(0.2)
    
    for items in auctions:
        
        title = items.find_element(By.TAG_NAME, 'h4')
        date = items.find_element(By.CLASS_NAME, 'chr-heading-xs-sans')
        auction_type = items.find_element(By.CLASS_NAME, 'chr-event-tile__subtitle')
        auction_total = items.find_elements(By.CLASS_NAME, 'chr-body-s')
        location = items.find_element(By.CLASS_NAME, 'chr-label-s')
        URL = items.find_element(By.TAG_NAME, 'a')
        
        time.sleep(0.05)
        
        title = items.find_element(By.TAG_NAME, 'h4')
        date = items.find_element(By.CLASS_NAME, 'chr-heading-xs-sans')
        auction_type = items.find_element(By.CLASS_NAME, 'chr-event-tile__subtitle')
        auction_total = items.find_elements(By.CLASS_NAME, 'chr-body-s')
        location = items.find_element(By.CLASS_NAME, 'chr-label-s')
        URL = items.find_element(By.TAG_NAME, 'a')
    
        Auction_titles.append(title.text)
        Auction_date.append(date.text)
        Auction_type.append(auction_type.text)
        Auction_sale_total.append(auction_total[-1].text)
        Auction_location.append(location.text)
        Auction_URL.append(URL.get_attribute('href'))

        
    

In [29]:
CookieCloser()
ClickEmail()

for i in range(10):
    n = random.random()
    time.sleep(n)
    try:
        
        #Tries to filter the search results
        try:
            click_catagories()
        except:
            pass
        try:
            click_fine_art()
        except:
            pass
        try:
            click_prints()
        except:
            pass
        
        #once filtered extract the data
        extract_General_Auction_Data()
        time.sleep(0.3)
    finally:
        click_next_page()
        time.sleep(1)
        driver.refresh()

MaxRetryError: HTTPConnectionPool(host='localhost', port=59975): Max retries exceeded with url: /session/45ede89359b18ccdab2c1d2e57f37ef9/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff0d14741c0>: Failed to establish a new connection: [Errno 61] Connection refused'))

## Cleaning :

In [ ]:
Location = []
for items in Auction_location:
    items = items.replace("EVENT LOCATION\n", "")
    Location.append(items)
    
Type = []
for items in Auction_type:
    items = items.replace("EVENT TYPE\n", "")
    items = items.replace("| CLOSED", "")
    Type.append(items)
    
Date = []
for items in Auction_date:
    items = items.replace("Event Date\n", "")
    Date.append(items)

In [ ]:
df = pd.DataFrame(list(zip(Auction_titles, Date, Type, Auction_sale_total, Location, Auction_URL)),
               columns =['Title', 'Date', 'Type', 'Total Sales', 'Location', 'URL'])

df.head(10)

In [ ]:
# df.to_csv("Christies_Auction_Details.csv", index = False)